In [ ]:
#corpus? - Set of words
#1. Convert NL -> Latent representation (vectors) - LM (BERT) (Masked Language Modelling)
#2. LM - Generate words (CHAT-GPT) - Generate next word (Given N words)

In [ ]:
#[hello, work, do , perform, task]
#Its a great morning Laxmi says ____hello____ 1. Generative models - Hallucinate, words only in training corpus

In [3]:
import numpy as np
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM , Embedding
from tensorflow.keras.preprocessing.text import Tokenizer

In [4]:
#smallest unit of a paragraph - word - Tokens
import re
def load_doc(file):
  #open in read mode
  file = open(file,'r')
  #reading text into the object
  texts = file.read()
  #close the connection
  file.close()
  return texts

In [7]:
text = "Hello how are y-==--=ouu 0*#@&&((@) doing"
doc = re.sub(r'[^\w\s]','',text)
doc

'Hello how are youu 0 doing'

In [8]:
paragraph = "The birch canoe slid on the smooth planks. Glue the sheet to the dark blue background"

In [10]:
paragraph.split('.')[0].split(' ')

['The', 'birch', 'canoe', 'slid', 'on', 'the', 'smooth', 'planks']

In [13]:
#INCEPTEZ - inceptez
def clean_doc(doc):
  doc = re.sub(r'[^\w\s]','',doc)
  #word tokenization
  tokens = doc.split()
  tokens = [word for word in tokens if word.isalpha()]
  #converting to lower case
  tokens = [word.lower() for word in tokens]
  return tokens

In [ ]:
clean_doc(paragraph)

In [32]:
doc = load_doc("republic_sequences.txt")
doc = doc[:1000000]

In [33]:
tokens = clean_doc(doc)

In [34]:
tokens[:10]

['introduction',
 'and',
 'analysis',
 'the',
 'republic',
 'of',
 'plato',
 'is',
 'the',
 'longest']

In [35]:
#train - 50 words and predict 51st word
length = 50+1
sequences = list()
for i in range(length,len(tokens)):
  #make the 50 sequencs - 0-51
  seq = tokens[i-length:i]
  line = " ".join(seq)
  sequences.append(line)

In [36]:
print("The total no. of sequences are {}".format(len(sequences)))

The total no. of sequences are 179800


In [37]:
#Encoding sequences
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sequences)

In [39]:
tokenizer.word_index

{'the': 1,
 'of': 2,
 'and': 3,
 'in': 4,
 'is': 5,
 'to': 6,
 'which': 7,
 'a': 8,
 'or': 9,
 'are': 10,
 'as': 11,
 'be': 12,
 'by': 13,
 'state': 14,
 'not': 15,
 'have': 16,
 'plato': 17,
 'has': 18,
 'justice': 19,
 'more': 20,
 'at': 21,
 'an': 22,
 'republic': 23,
 'with': 24,
 'but': 25,
 'may': 26,
 'from': 27,
 'he': 28,
 'all': 29,
 'other': 30,
 'was': 31,
 'first': 32,
 'into': 33,
 'on': 34,
 'been': 35,
 'for': 36,
 'his': 37,
 'than': 38,
 'only': 39,
 'philosophy': 40,
 'there': 41,
 'life': 42,
 'like': 43,
 'we': 44,
 'this': 45,
 'it': 46,
 'one': 47,
 'them': 48,
 'truth': 49,
 'that': 50,
 'work': 51,
 'no': 52,
 'cp': 53,
 'books': 54,
 'who': 55,
 'him': 56,
 'great': 57,
 'same': 58,
 'world': 59,
 'well': 60,
 'had': 61,
 'mind': 62,
 'own': 63,
 'some': 64,
 'whole': 65,
 'idea': 66,
 'about': 67,
 'higher': 68,
 'writings': 69,
 'second': 70,
 'argument': 71,
 'they': 72,
 'ideas': 73,
 'such': 74,
 'any': 75,
 'would': 76,
 'knowledge': 77,
 'most': 78,
 'd

In [40]:
len(tokenizer.word_index)+1

1026

In [41]:
vocab_size = len(tokenizer.word_index)+1

In [42]:
sequences = tokenizer.texts_to_sequences(sequences)

In [ ]:
sequences[0]

In [44]:
sequences = np.array(sequences)
X,y = sequences[:,:-1],sequences[:,-1]
y = to_categorical(y, num_classes = vocab_size)
seq_length = X.shape[1]

In [45]:
model= Sequential()

In [46]:
model.add(Embedding(vocab_size, 50, input_length = seq_length))
model.add(LSTM(100, return_sequences = True))
model.add(LSTM(100))
model.add(Dense(vocab_size,activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 50, 50)            51300     
                                                                 
 lstm (LSTM)                 (None, 50, 100)           60400     
                                                                 
 lstm_1 (LSTM)               (None, 100)               80400     
                                                                 
 dense (Dense)               (None, 1026)              103626    
                                                                 
Total params: 295,726
Trainable params: 295,726
Non-trainable params: 0
_________________________________________________________________


In [47]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam',metrics = ['accuracy'])

In [48]:
model.fit(X,y,batch_size = 128, epochs = 10)

Epoch 1/10
1405/1405 [==============================] - 49s 28ms/step - loss: 4.9398 - accuracy: 0.1650
Epoch 2/10
1405/1405 [==============================] - 16s 11ms/step - loss: 3.8321 - accuracy: 0.2647
Epoch 3/10
1405/1405 [==============================] - 16s 12ms/step - loss: 3.0103 - accuracy: 0.3717
Epoch 4/10
1405/1405 [==============================] - 14s 10ms/step - loss: 2.2657 - accuracy: 0.5225
Epoch 5/10
1405/1405 [==============================] - 14s 10ms/step - loss: 1.6212 - accuracy: 0.6893
Epoch 6/10
1405/1405 [==============================] - 14s 10ms/step - loss: 1.1604 - accuracy: 0.8145
Epoch 7/10
1405/1405 [==============================] - 15s 10ms/step - loss: 0.8374 - accuracy: 0.8903
Epoch 8/10
1405/1405 [==============================] - 15s 11ms/step - loss: 0.6325 - accuracy: 0.9271
Epoch 9/10
1405/1405 [==============================] - 16s 11ms/step - loss: 0.5038 - accuracy: 0.9434
Epoch 10/10
1405/1405 [==============================] - 14s 10m

In [ ]:
:50 - 51st output
:5  - 6th
#padding
000000...44th -----  -> Pre
----- : 000000...50th -> Post

In [ ]:
for x in y_ANNprediction:
    y_ANN_prediction_classes.append(np.argmax(x))

#Conver the list back to a numpy arrach
y_ANN_prediction_classes_arr = np.array(y_ANN_prediction_classes)

In [74]:
def generate_seq(model, tokenizer, seq_length,in_text, n_words):
  for _ in range(n_words):
    encoded = tokenizer.texts_to_sequences([in_text])
    encoded = keras.preprocessing.sequence.pad_sequences(encoded, maxlen = seq_length, truncating = 'pre')
    y_prediction_classes = []
    yhat_ = model.predict(encoded,verbose = 0)
    for x in yhat_:
      y_prediction_classes.append(np.argmax(x))
    yhat = np.array(y_prediction_classes)
    out_word = ''
    for word, index in tokenizer.word_index.items():
      if index == yhat:
        out_word = word
        break
    in_text+=" "+out_word
  return in_text

In [75]:
in_text = "Hi, I am taking classes here at inceptez"

In [78]:
generated = generate_seq(model,tokenizer,seq_length, in_text,10)

In [79]:
generated

'Hi, I am taking classes here at inceptez some elements confirmed idea under poetry had better be included'